In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

In [ ]:
# carregar os dados do CSV
df = pd.read_csv("dados_3_meses.csv")

In [19]:
# calcular a energia gerada (Wh) = irradiação * tempo (em horas) * área do painel * eficiência

tempo_horas = 0.25 # 15 minutos
area_painel = 2 # m2 - área média 

def eficiencia_energetica(status):
    if status == "Operando...":
        return 0.18
    elif status == "Falha":
        return 0.05
    else:
        return 0.0

energia_por_registro = df.apply(lambda x: round(x['irradiacao_solar'] * eficiencia_energetica(x['status_painel']) * tempo_horas * 1.6), axis=1) 

# Verificar o quanto de energia o painel gera por dia (Wh), com base na eficiência energética do painel solar

energia_gerada_dia = energia_por_registro.groupby(df["data"]).sum()
print(energia_gerada_dia)

data
2025-02-12       0
2025-02-13    1916
2025-02-14    1902
2025-02-15    1855
2025-02-16    1830
              ... 
2025-05-09    2108
2025-05-10    1722
2025-05-11    1825
2025-05-12    1694
2025-05-13    1769
Length: 91, dtype: int64


In [ ]:
# Dias mais produtivos (maior energia gerada)

In [ ]:
# Desempenho do painel durante o dia (quais horários o desempenho do painel é mais eficiente)

In [ ]:
# Em quais horários do dia há maior probabilidade de falhas

In [ ]:
# Relação entre índice de falhas e temperatura do painel

In [ ]:
# Faixa de temperatura ambiental em que o painel apresenta menos falhas